In [269]:
import sqlalchemy
import pandas as pd
import pickle as pkl
import numpy as np
import json

from fbprophet import Prophet
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [270]:
# get url to create engine
hostName = 'project-kojak1.cekiwi9udsce.us-east-2.rds.amazonaws.com'
user = 'Rishi'
pw = 'password' # incorrect password
port = '5432'
database = 'dbsales'
url = 'postgres://' + user + ':' + pw + '@' + hostName + ':' + port + '/' + database

In [271]:
# create engine
engine = sqlalchemy.create_engine(url)

In [272]:
# filter df based on store and item
def getDFofStoreItem(storeNum, itemNum):    
    myStore = pd.read_sql_query('SELECT * FROM "Sales" WHERE "store" = {} AND "item" = {}'.format(storeNum, itemNum), 
                                engine)
    myStore.date = pd.to_datetime(myStore.date)
    myStore = myStore.reset_index()
    myStore = myStore.drop(columns=['store', 'item', 'level_0', 'index'])
    myStore = myStore.sort_values('date')
    myStore.columns = ['ds', 'y']
    
    return myStore

In [273]:
# find percent error
def getPercentError(current, test):
    return (current-test)/current

In [274]:
# test different train/test splits to make sure models remain consistent
def testDF(storeNum, itemNum, trainProportions=[0.5, 0.67, 0.75, 0.9]):
    storeItemDF = getDFofStoreItem(storeNum, itemNum)
    
    rmse = []
    for proportion in trainProportions:
        num = int(proportion*len(storeItemDF))
        train = storeItemDF[:num]
        test = storeItemDF[num:]
        
        prophet = Prophet()
        prophet.add_seasonality(name='yearly', period=365, fourier_order=10)
        prophet.fit(train)
        future = prophet.make_future_dataframe(periods=len(test))
        forecast = prophet.predict(future)
        rmse.append(np.sqrt(mean_squared_error(test['y'], forecast.yhat[-len(test):])))
    
    for root in rmse:
        for check in rmse:
            if abs(getPercentError(root, check)) > 0.2:
                return False
    return True 

In [305]:
# create forecast using fbprophet for specified amount of days and return df
def getForecastForStoreAndItemPandas(storeNum, itemNum, numDays=30, totalInfo=False, printSummary=True):
    df = getDFofStoreItem(storeNum, itemNum)
    testValue = testDF(storeNum, itemNum)
    prophet = Prophet()
    prophet.add_seasonality(name='yearly', period=365, fourier_order=10)
    prophet.fit(df)
    future = prophet.make_future_dataframe(periods=numDays)
    forecast = prophet.predict(future)
    
    forecast = forecast.rename(columns={'ds':'Date', 'yhat':'Predicted Sales'})
    forecast = forecast.set_index('Date')
    pred = forecast.iloc[-numDays:]
    
    if printSummary:   
        total = int(pred['Predicted Sales'].sum())
        margin = int((pred['yhat_upper'].sum() - pred['yhat_lower'].sum())/2)
        mySummary = 'The projected total sales for Item ' + str(itemNum) + ' at Store ' + str(storeNum) + ' for ' + str(numDays) + ' days is about ' + str(total) + ' with a margin of error of ' + str(margin)
    
    if not testValue:
        print('These predictions may not be completely accurate')

    if totalInfo and printSummary:
        return pred, mySummary
    elif totalInfo and not printSummary:
        return pred, ''
    elif not totalInfo and printSummary:
        return pred['Predicted Sales'], mySummary
    else:
        return pred['Predicted Sales'], ''

In [306]:
# create forecast using fbprophet for specified amount of days and return json
def getForecastForStoreAndItemJSON(storeNum, itemNum, numDays=30, totalInfo=False, printSummary=True):
    if printSummary == True:
        df, summary = getForecastForStoreAndItemPandas(storeNum, itemNum, numDays, totalInfo=totalInfo, printSummary=printSummary)
        df.index = df.index.astype(str)
        salesJSON = df.to_json(orient='index')
        return salesJSON, summary
    else:
        df, blank = getForecastForStoreAndItemPandas(storeNum, itemNum, numDays, totalInfo=totalInfo, printSummary=printSummary)
        df.index = df.index.astype(str)
        salesJSON = df.to_json(orient='index')
        return salesJSON, blank

In [307]:
# final function in pipeline - returns final product
def getForecast(storeNum, itemNum, numDays=30, totalInfo=False, printSummary=True, JSON=False):
    if JSON:
        return getForecastForStoreAndItemJSON(storeNum, itemNum, numDays, totalInfo, printSummary)
    else:
        return getForecastForStoreAndItemPandas(storeNum, itemNum, numDays, totalInfo, printSummary)

In [ ]:
myForecast = getForecast(7, 20, numDays=15, totalInfo=True)